In [563]:
import sys
sys.path.append('../')

from IPython.core.display import display
import numpy as np
import pandas as pd

In [564]:
# import node embeddings
df = pd.read_csv(
    "./movie_ratings/movie_ratings_alpha-0.1_beta-0.1_ws-10_neg-5_lr-0.025_icom-107_ind-107_k-300_ds-0.0.txt",
    sep="\t| ",
    header=None
)
df = df.rename(columns={0: 'node'})
df.set_index(['node'], inplace=True)

/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


In [565]:
# import ratings
ratings = pd.read_csv("movie_ratings/movie_ratings_unique.csv")
ratings.set_index(['userId', 'movieId'], inplace=True)
# import test
test = pd.read_csv("movie_ratings/movie_ratings_test.csv", index_col=0)
test.set_index(['userId', 'movieId'], inplace=True)

In [566]:
# node embeddings
display(df.head())
display(df.describe())

,1,2,3,4,5,6,7,8,9,10,...,119,120,121,122,123,124,125,126,127,128
node,,,,,,,,,,,,,,,,,,,,,
1,0.084126,-0.158210,0.055267,0.064245,0.119639,-0.457690,0.151260,-0.188201,0.306511,-0.171590,...,0.151671,-0.428449,0.384514,0.115550,-0.281860,-0.375460,-0.212332,-0.118784,-0.340415,-0.532844
2,0.189232,0.368353,-0.021794,-0.180266,-0.164473,0.112753,-0.024415,-0.059802,-0.087151,-0.377635,...,-0.047132,-0.445814,-0.271653,0.155094,-0.182624,-0.277436,-0.269682,0.209155,-0.150452,-0.109370
3,0.202998,0.574760,0.186271,-0.307789,-0.409419,-0.294299,0.235681,0.074319,0.207355,0.147867,...,-0.323176,0.062072,0.202059,0.245980,-0.446667,0.251941,-0.110601,0.085919,-0.233315,-0.669942
4,0.257892,0.118488,-0.068176,0.164954,-0.099770,-0.385578,-0.176414,0.486536,0.246525,-0.154400,...,0.002303,-0.457141,0.298918,-0.118033,-0.224613,-0.202845,-0.158876,-0.008919,0.143959,-0.383050
5,-0.087077,-0.112597,-0.015801,-0.042038,-0.051923,-0.367648,0.079132,0.445809,0.080819,-0.062714,...,-0.214951,0.092153,-0.000992,0.378648,-0.211254,-0.131860,-0.561901,0.055496,-0.243451,-0.627809


,1,2,3,4,5,6,7,8,9,10,...,119,120,121,122,123,124,125,126,127,128
count,6835.000000,6835.000000,6835.000000,6835.000000,6835.000000,6835.000000,6835.000000,6835.000000,6835.000000,6835.000000,...,6835.000000,6835.000000,6835.000000,6835.000000,6835.000000,6835.000000,6835.000000,6835.000000,6835.000000,6835.000000
mean,0.078405,0.048297,0.115129,-0.092659,-0.126263,-0.197804,0.051906,0.002985,0.006459,0.204804,...,-0.074648,0.023204,0.067371,0.130251,-0.136144,-0.165764,-0.095447,0.061687,-0.151986,-0.151810
std,0.201490,0.179932,0.210509,0.169755,0.175618,0.182212,0.184918,0.167601,0.187386,0.186090,...,0.191998,0.212256,0.197551,0.203540,0.173769,0.179140,0.196246,0.182510,0.187898,0.171702
min,-0.616688,-0.755005,-0.706490,-0.834755,-0.737403,-0.766144,-0.789684,-0.700273,-0.618574,-0.540283,...,-0.769387,-0.736748,-0.644136,-0.551041,-0.839786,-0.823649,-0.816942,-0.598190,-0.915370,-0.724223
25%,-0.057553,-0.071159,-0.002983,-0.195428,-0.238303,-0.319454,-0.064229,-0.107144,-0.124308,0.083389,...,-0.204798,-0.128299,-0.079646,-0.012380,-0.259488,-0.281507,-0.223465,-0.069834,-0.274472,-0.265247
50%,0.074405,0.036614,0.132743,-0.087690,-0.127753,-0.207378,0.058246,0.007376,-0.000737,0.212001,...,-0.067939,0.017214,0.079114,0.134905,-0.134756,-0.170685,-0.103659,0.051395,-0.159008,-0.149449
75%,0.219811,0.161051,0.250212,0.014769,-0.007301,-0.077391,0.170863,0.116817,0.124352,0.320747,...,0.063769,0.176387,0.207024,0.259615,-0.007556,-0.043371,0.026951,0.193507,-0.035574,-0.037947
max,0.845680,0.709137,0.809617,0.577804,0.692225,0.584818,0.772504,0.562082,0.772284,0.858027,...,0.566391,0.927946,0.729018,0.917395,0.429429,0.461738,0.791065,0.711909,0.966309,0.631978


In [567]:
# ratings df
display(ratings.head())
#display(ratings.describe())

# test df
display(test.head())
#display(test.describe())

Unnamed: 0  rating   timestamp
userId movieId                                
1      1031              0     2.5  1260759144
       2029              1     3.0  1260759179
       2061              2     3.0  1260759182
       2129              3     2.0  1260759185
       2172              4     4.0  1260759205

,,rating,timestamp,is_test
userId,movieId,,,
1,2172,4.0,1260759205,True
2,1017,5.0,835355681,True
3,1318,5.0,1298862121,True
4,1034,5.0,949919556,True
5,1597,5.0,1163373711,True


In [568]:
# remove test edges from ratings
print("len(ratings): ", len(ratings))
print("len(test): ", len(test))
ratings = ratings[~ratings.index.isin(test.index)]
print("len(ratings): ", len(ratings))

len(ratings):  100004
len(test):  671
len(ratings):  99333


In [569]:
def emb_from_row(row):
    """get node embedding np array from df row"""
    return row.to_numpy().flatten()[1:]

def emb_distance(emb1, emb2):
    """computes the euclidean distance between two node embeddings"""
    return np.linalg.norm(emb1)

In [570]:
# get dataframe with movie embeddings
print("len(df): ", len(df))
userIds = ratings.index.get_level_values(level="userId").unique()
print("len(userIds): ", len(userIds))
df_movies = df[~df.index.isin(userIds)]
print("len(df_movies): ", len(df_movies))

len(df):  6835
len(userIds):  671
len(df_movies):  6167


In [574]:
# compute Top-N lists for test dataset
# for each testing row (for each user)
for i, test_row in test.iterrows():
    userId = i[0]
    user_emb = emb_from_row(df[userId])

    # get new (unrated) movies
    print("len(df_movies): ", len(df_movies))
    movies_watched = ratings.query(f"userId == {userId}").index.get_level_values("movieId")
    print("len(movies_watched): ", len(movies_watched))
    df_movies_unwatched = df_movies[~df_movies.index.isin(movies_watched)]
    print("len(df_movies_unwatched): ", len(df_movies_unwatched))

    # for each node embedding get distance to user embedding
    distances = []
    for j, node_row in df.iterrows():
        node_emb = emb_from_row(node_row)
        dist = emb_distance(user_emb, node_emb)
        print(dist)
        distances.append()

    break

len(df_movies):  6167
len(movies_watched):  19
len(df_movies_unwatched):  6148
17.873420243310818


TypeError: append() takes exactly one argument (0 given)